In [1]:
# must run on activated terminal
# i.e source activate py310
!pip install --user transformers[torch] datasets evaluate
!pip install --user torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

zsh:1: no matches found: transformers[torch]


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
ERROR: Could not find a version that satisfies the requirement torch==1.13.1+cu116 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.13.1+cu116


In [6]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=391a41948e2cd4f4c631053d7d55150c7690e30583bdc544f47cfce3bbc7abee
  Stored in directory: /home/jupyter/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
feature_df

,full_caption,username,Openness,Conscientiousness,Extroversion,Agreeableness,Neuroticism
59,caption did you know henry ford made the 8 hou...,36mktg,2,2,2,2,0
69,caption so clean smiling_face_with_heart-eyesr...,6staroileaters,0,2,1,1,1
26,caption through the night crescent_moon • maet...,740project,2,0,0,0,1
57,caption take your social media accounts out of...,_boostfruit.com_,2,0,2,1,1
10,caption appreciate you excellencealways folded...,_cloutfinder,2,0,2,1,1
...,...,...,...,...,...,...,...
71,"caption none, is of type image, has 1 comments...",wipmarketing,2,2,2,2,0
14,caption enjoy this video while i disappear for...,xcarmeeen,0,0,2,0,2
5461,"caption power back.#fanggang | xfl, is of type...",xflvipers,0,2,2,2,0
67,caption summeeeeer! all the hard work of clean...,yogabody_uk,2,2,2,2,0


In [ ]:
feature_df[feature_df['Openness']==0]['full_caption']

69      caption so clean smiling_face_with_heart-eyesr...
5490    caption best day of my life 🫶heart_suit️jetzt ...
22      caption memorial day weekenTongue_sticking_out...
5484    caption mission accomplished. mold broken. i l...
47      caption anyone got a pre-order on one of these...
                              ...                        
108     caption logo variations thinking_face.....#dig...
24      caption wah taj!!#taj 2nd season#congratulatio...
136     caption it’s in our dna.big love,tbx, is of ty...
14      caption enjoy this video while i disappear for...
5461    caption power back.#fanggang | xfl, is of type...
Name: full_caption, Length: 66, dtype: object

In [ ]:
feature_df.to_parquet("feature.parquet")

In [ ]:
pd.options.display.max_colwidth=100
feature_df[feature_df['Openness']==0].full_caption.iloc[1]

'caption best day of my life \U0001faf6heart_suit️jetzt überall erhältlich firehundred_pointsfolded_hands•clip by: filmschieber_agency > _luck3 track by: komacasper x lucawolf_ #komacasper #vocaltekkno #vocaltekknoworldwide #lucawolf #ssb #festival #rave, is of type video, has 37 comments and 1804.0 likescaption sputnik springbreak 2023 es war absolut grandios smiling_face_with_heart-eyeshundred_pointsdankeschön für dieses unvergessliche erlebnis, ihr habt es zu dem gemacht was es war 6000 menschen die bock auf vocaltekkno hatten! jeder der dabei war darf sich persönlich von mir gedrückt fühlen heart_suit️folded_handsdankeschön bis 2024 double_exclamation_mark️smiling_face_with_halohundred_points•video by: filmschieber_agency > _luck3 #komacasper #vocaltekkno #vocaltekknoworldwide #sputnik #festival #ssb #julianluckeproductions #rave, is of type video, has 65 comments and 3522.0 likescaption der sommer ist da und hier sind meine dates für juni und juli 2023 fire\U0001faf6wir sehen uns 

In [ ]:
feature_df['Openness'].value_counts()

2.0    200
0.0     66
Name: Openness, dtype: int64

In [ ]:
feature_df['Conscientiousness'].value_counts()

0.0    61
2.0     1
Name: Conscientiousness, dtype: int64

In [ ]:
feature_df['Extroversion'].value_counts()

2    229
0     27
1     10
Name: Extroversion, dtype: int64

In [ ]:
feature_df['Agreeableness'].value_counts()

2    139
0     82
1     45
Name: Agreeableness, dtype: int64

In [ ]:
feature_df['Neuroticism'].value_counts()

0    194
1     45
2     27
Name: Neuroticism, dtype: int64

In [11]:
from collections import defaultdict
from transformers import AutoTokenizer
# using disilroberta as a quick baseline model
# https://huggingface.co/docs/transformers/tasks/sequence_classification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

focused_trait = "Neuroticism"


def tokenize_input(examples):
    return tokenizer(examples["full_caption"])

def group_texts(examples):
    block_size = 128    
    
    result= defaultdict(list)
    for i, input_id  in enumerate(examples['input_ids']):
        attn_mask = examples['attention_mask'][i]
        total_length = len(input_id)
        if total_length > block_size:
            total_length = (total_length // block_size) * block_size
        num_chunks = total_length // block_size
        result['input_ids'].extend([input_id[i : i + block_size] for i in range(0, total_length, block_size)])
        result['attention_mask'].extend([attn_mask[i : i + block_size] for i in range(0, total_length, block_size)])
        result['Openness'].extend([examples['Openness'][i] for _ in range(num_chunks)])
        result['Conscientiousness'].extend([examples['Conscientiousness'][i] for _ in range(num_chunks)])
        result['Extroversion'].extend([examples['Extroversion'][i] for _ in range(num_chunks)])
        result['Agreeableness'].extend([examples['Agreeableness'][i] for _ in range(num_chunks)])
        result['Neuroticism'].extend([examples['Neuroticism'][i] for _ in range(num_chunks)])    

    return result


from datasets import Dataset
import pandas as pd 
all_five = ['Openness', 'Conscientiousness','Extroversion', 'Agreeableness', 'Neuroticism']
feature_df = pd.read_parquet("feature.parquet")
feature_df = feature_df.dropna()

d = {0.5:1,1.0:2,0:0}
for column in all_five:
    feature_df[column] = feature_df[column].map(d)
    feature_df.astype({column:'int32'}).dtypes

all_five.remove(focused_trait)

ds = Dataset.from_dict(feature_df.to_dict('list'))
tokenized_ds = ds.map(tokenize_input, batched=True, num_proc=4, remove_columns=['full_caption','username'])
lm_dataset_ig = tokenized_ds.map(group_texts, num_proc=4, batched=True)
lm_dataset_ig = lm_dataset_ig.shuffle(seed=42)
lm_dataset_ig = lm_dataset_ig.rename_column(focused_trait, "label")
lm_dataset_ig = lm_dataset_ig.remove_columns(all_five)
lm_dataset_ig = lm_dataset_ig.train_test_split(test_size=0.2)
lm_dataset_ig

Map (num_proc=4):   0%|          | 0/266 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (13472 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4864 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3007 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (25039 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/266 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 8977
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2245
    })
})

In [ ]:
lm_dataset_ig['train']['label']

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,


### Dataset IO

In [ ]:
lm_dataset_ig.save_to_disk("distilbert_base_block128_train_test_split_tokenized_feature")

Saving the dataset (0/1 shards):   0%|          | 0/8977 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
lm_dataset_ig = load_from_disk("distilbert_base_block128_train_test_split_tokenized_feature")

In [ ]:
# rename Agreeableness to label

In [ ]:
lm_dataset_ig['train']['label']

[2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,


In [12]:
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    print(f"predictions:{predictions}, item_freq:{np.unique(predictions, return_counts=True)}.")
    print(f"labels:{labels}, item_freq:{np.unique(labels, return_counts=True)}.")
    accuracy_measurement =  accuracy.compute(predictions=predictions, references=labels)
    precision_measurement =  precision.compute(predictions=predictions, references=labels, average='micro')
    recall_measurement =  recall.compute(predictions=predictions, references=labels, average='micro')
    print(f"Accuracy:{accuracy_measurement}, Precision:{precision_measurement}, Recall:{recall_measurement}")
    return precision_measurement

id2label = {0: "LOW", 1: "VAGUE", 2: "HIGH"}
label2id = {"LOW": 0, "VAGUE":1, "HIGH": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)
import os 
os.environ["WANDB_PROJECT"]="upwork-ocean-classification"
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="false"

training_args = TrainingArguments(
    output_dir=f"distilbert-base-uncased-{focused_trait}-classification-shuffled",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
    report_to="wandb",
    push_to_hub=False,
    run_name=focused_trait
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_ig["train"],
    eval_dataset=lm_dataset_ig["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [13]:
import wandb
trainer.train()
wandb.finish()
trainer.save_state()
trainer.save_model()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision
1,0.522900,0.325799,0.885969
2,0.258100,0.304947,0.909577
3,0.152200,0.251792,0.938085
4,0.083800,0.260075,0.946548
5,0.042900,0.254726,0.946993


predictions:[0 0 0 ... 1 0 0], item_freq:(array([0, 1, 2]), array([1845,  360,   40])).
labels:[0 0 0 ... 1 0 0], item_freq:(array([0, 1, 2]), array([1733,  356,  156])).
Accuracy:{'accuracy': 0.8859688195991091}, Precision:{'precision': 0.8859688195991091}, Recall:{'recall': 0.8859688195991091}
predictions:[0 0 0 ... 1 0 0], item_freq:(array([0, 1, 2]), array([1816,  375,   54])).
labels:[0 0 0 ... 1 0 0], item_freq:(array([0, 1, 2]), array([1733,  356,  156])).
Accuracy:{'accuracy': 0.9095768374164811}, Precision:{'precision': 0.9095768374164811}, Recall:{'recall': 0.9095768374164811}
predictions:[0 0 0 ... 1 0 0], item_freq:(array([0, 1, 2]), array([1826,  283,  136])).
labels:[0 0 0 ... 1 0 0], item_freq:(array([0, 1, 2]), array([1733,  356,  156])).
Accuracy:{'accuracy': 0.9380846325167038}, Precision:{'precision': 0.9380846325167038}, Recall:{'recall': 0.9380846325167038}
predictions:[0 0 0 ... 1 0 0], item_freq:(array([0, 1, 2]), array([1779,  317,  149])).
labels:[0 0 0 ... 1 0

eval/loss,█▆▁▂▁
eval/precision,▁▄▇██
eval/runtime,▅█▄▂▁
eval/samples_per_second,▄▁▅▇█
eval/steps_per_second,▄▁▅▇█
train/epoch,▁▁▃▃▄▅▆▆▇██
train/global_step,▁▁▃▃▄▅▆▆▇██
train/learning_rate,█▆▄▃▁
train/loss,█▄▃▂▁
train/total_flos,▁
train/train_loss,▁


## inference!

In [57]:
import pandas as pd
inference_dataset = pd.read_parquet("prediction_feature.parquet")
prediction_dataset = pd.read_parquet("concat_feature.parquet")

In [4]:
inference_dataset

,full_caption
ig_username,
007offmason,caption out now on all platformsglobe_showing_...
0g.ed1tz,"caption none, is of type image, has 6 comments..."
1000taraneh,caption .«لج و لج بازی» با صدای زنده یاد پوران...
1000thingstodoindenver,caption opening day specials: here are some fu...
101_molly,caption yessirr let’s go!! firefire🩸red_heart️...
...,...
zuber_zooby,caption titanic movie explained in hindi face_...
zuraidajardine,caption unhappy fathers day 🩶today can hit dif...
zuri_baby,caption never in a million years did i think i...


In [77]:
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoTokenizer
from collections import defaultdict

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

def tokenize_input(examples):
    return tokenizer(examples["full_caption"], padding=True)

def group_texts_for_inference(examples):
    block_size = 512    
    
    result= defaultdict(list)
    for i, input_id  in enumerate(examples['input_ids']):
        attn_mask = examples['attention_mask'][i]
        total_length = len(input_id)
        if total_length < block_size:
            result['input_ids'].extend([input_id])
            result['attention_mask'].extend([attn_mask])
            result['ig_username'].extend([examples['ig_username'][i]])
            continue                        
        if total_length > block_size:
            total_length = (total_length // block_size) * block_size
        num_chunks = total_length // block_size
        with open ("log.log", "a") as f_out:
            f_out.write(f"{i} {total_length} {block_size} {num_chunks}\n")
        result['input_ids'].extend([input_id[i : i + block_size] for i in range(0, total_length, block_size)])
        result['attention_mask'].extend([attn_mask[i : i + block_size] for i in range(0, total_length, block_size)])
        result['ig_username'].extend([examples['ig_username'][i] for _ in range(num_chunks)])
    return result


#ds = Dataset.from_dict(inference_dataset.reset_index().to_dict('list'))
#tokenized_ds = ds.map(tokenize_input, batched=True, num_proc=4, remove_columns=['full_caption'])
lm_dataset_ig = tokenized_ds.map(group_texts_for_inference, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/14240 [00:00<?, ? examples/s]

In [2]:
type(lm_dataset_ig)

datasets.arrow_dataset.Dataset

In [1]:
from datasets import load_from_disk
lm_dataset_ig = load_from_disk("grouped_prediction_dataset")

In [79]:
lm_dataset_ig.save_to_disk("grouped_prediction_dataset")

Saving the dataset (0/3 shards):   0%|          | 0/409880 [00:00<?, ? examples/s]

In [11]:
tokenized_ds.save_to_disk("tokenized_prediction_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/14240 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoTokenizer, DistilBertModel, DistilBertForSequenceClassification
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lm_dataset_ig_gpu = lm_dataset_ig.with_format("torch", device=device)

In [3]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
focused_trait = "Agreeableness"
ft_model = AutoModelForSequenceClassification.from_pretrained(f"distilbert-base-uncased-{focused_trait}-classification-shuffled")
ft_model = ft_model.to(device)

2023-07-01 02:08:50.618016: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-01 02:08:53.296281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-07-01 02:08:53.296427: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [ ]:
import torch
import pandas as pd
all_predictions = pd.DataFrame()
batch_size = 40
with torch.no_grad():
    for i in range(0, len(lm_dataset_ig_gpu), batch_size):
        predictions = ft_model(
            input_ids=lm_dataset_ig_gpu['input_ids'][i: i+batch_size],
            attention_mask=lm_dataset_ig_gpu['attention_mask'][i: i+batch_size]
        ).logits.argmax(axis=1)
        predictions_df_current_batch = pd.DataFrame(
            {
                "usernames":lm_dataset_ig['ig_username'][i: i+batch_size], 
                "predictions":predictions.to('cpu')}
            )
        pd.concat([all_predictions, predictions_df_current_batch])
        if i%10==0:
            print(i)
    

0
40
80
120
160
200
240
280
320
360
400
440
480
520
560
600
640
680
720
760
800
840
880
920
960
1000
1040
1080
1120
1160
1200
1240
1280
1320
1360
1400
1440
1480
1520
1560
1600
1640
1680
1720
1760
1800
1840
1880
1920
1960
2000
2040
2080
2120
2160
2200
2240
2280
2320
2360
2400
2440
2480
2520
2560
2600
2640
2680
2720
2760
2800
2840
2880
2920


In [ ]:
## Post process

In [51]:
predictions_df = predictions_df.groupby('usernames').agg([list, set])
predictions_df

predictions           
                                          list        set
usernames                                                
2jpantoja                               [2, 0]     {0, 2}
_moneybo                             [2, 2, 2]        {2}
_ts97__            [2, 2, 2, 2, 2, 2, 2, 2, 2]        {2}
aless_capparelli               [0, 0, 0, 0, 0]        {0}
alexsaenz                         [2, 1, 1, 1]     {1, 2}
...                                        ...        ...
vngnc                       [2, 2, 2, 1, 1, 0]  {0, 1, 2}
waynebutlercomedy                 [2, 2, 2, 2]        {2}
woolimusic                              [1, 0]     {0, 1}
zairmontes            [2, 1, 0, 2, 2, 2, 2, 2]  {0, 1, 2}
zambezijuice                   [0, 0, 0, 0, 0]        {0}

[88 rows x 2 columns]

In [53]:
predictions_df.columns = predictions_df.columns.to_flat_index()

In [60]:
predictions_df = predictions_df.reset_index()

In [66]:
predictions_df['predictions_count'] = predictions_df.apply(lambda r: len(r[('predictions','list')]), axis=1)
predictions_df['predictions_unique_count'] = predictions_df.apply(lambda r: len(r[('predictions','set')]), axis=1)
predictions_df

,usernames,"(predictions, list)","(predictions, set)",predictions_count,predictions_unique_count
0,2jpantoja,"[2, 0]","{0, 2}",2,2
1,_moneybo,"[2, 2, 2]",{2},3,1
2,_ts97__,"[2, 2, 2, 2, 2, 2, 2, 2, 2]",{2},9,1
3,aless_capparelli,"[0, 0, 0, 0, 0]",{0},5,1
4,alexsaenz,"[2, 1, 1, 1]","{1, 2}",4,2
...,...,...,...,...,...
83,vngnc,"[2, 2, 2, 1, 1, 0]","{0, 1, 2}",6,3
84,waynebutlercomedy,"[2, 2, 2, 2]",{2},4,1
85,woolimusic,"[1, 0]","{0, 1}",2,2
86,zairmontes,"[2, 1, 0, 2, 2, 2, 2, 2]","{0, 1, 2}",8,3


In [144]:
predictions_df

,usernames,"(predictions, list)","(predictions, set)",predictions_count,predictions_unique_count,predictions_final
0,2jpantoja,"[2, 0]","{0, 2}",2,2,1
1,_moneybo,"[2, 2, 2]",{2},3,1,2
2,_ts97__,"[2, 2, 2, 2, 2, 2, 2, 2, 2]",{2},9,1,2
3,aless_capparelli,"[0, 0, 0, 0, 0]",{0},5,1,0
4,alexsaenz,"[2, 1, 1, 1]","{1, 2}",4,2,1
...,...,...,...,...,...,...
83,vngnc,"[2, 2, 2, 1, 1, 0]","{0, 1, 2}",6,3,2
84,waynebutlercomedy,"[2, 2, 2, 2]",{2},4,1,2
85,woolimusic,"[1, 0]","{0, 1}",2,2,1
86,zairmontes,"[2, 1, 0, 2, 2, 2, 2, 2]","{0, 1, 2}",8,3,2


In [143]:
predictions_df.predictions_final.value_counts()

2    49
1    27
0    12
Name: predictions_final, dtype: int64

In [140]:
#freq = np.bincount(predictions_df.iloc[87][('predictions', 'list')])
l = [1]
freq = np.bincount(l)
freq_index = np.flip(np.argsort(freq))

#l[winners[0]]
if len(freq) == 1:
    print(freq_index[0])
if freq[freq_index[0]] == freq[freq_index[1]]:
    print(1)
elif freq[freq_index[0]] > freq[freq_index[1]]:
    print(freq_index[0])

[0 1] [1 0]
1


In [141]:
import numpy as np
# take the one with largest frequency.
# when there is a draw (c0=c1=v2) (c0=c2) (c1=c2), (c0=c1) return vague.
def decision(row):
    freq = np.bincount(row[('predictions', 'list')])
    freq_index = np.flip(np.argsort(freq))
    if len(freq) == 1:
        return freq_index[0]
    elif freq[freq_index[0]] == freq[freq_index[1]]:
        return 1 # vague
    elif freq[freq_index[0]] > freq[freq_index[1]]:
        return freq_index[0]


        
predictions_df['predictions_final'] = predictions_df.apply(decision, axis=1)

## Example from 
https://huggingface.co/docs/transformers/tasks/language_modeling#preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    print(concatenated_examples.keys())
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [ ]:
from datasets import load_dataset

eli5 = load_dataset("eli5", split="train_asks[:5000]")
eli5 = eli5.flatten()
eli5 = eli5.train_test_split(test_size=0.2)
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Found cached dataset eli5 (/home/jupyter/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (693 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1311 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (651 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (982 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1483 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (618 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (801 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1161 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
tokenized_eli5.column_names

{'train': ['input_ids', 'attention_mask'],
 'test': ['input_ids', 'attention_mask']}

In [ ]:
#tokenized_eli5.keys()
sum(tokenized_ds['train'], [])

TypeError: can only concatenate list (not "dict") to list

In [ ]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])


In [ ]:
len(lm_dataset['train'][0]['input_ids'])

128

In [ ]:
tokenized_eli5['train']

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4000
})

In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

2023-06-07 19:41:22.795413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 19:41:25.997772: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-07 19:41:25.997902: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

RuntimeError: Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
/opt/conda/lib/python3.10/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: cudaGraphInstantiateWithFlags, version libcudart.so.11.0